# 5.Date and Time

Section 5.1: Parsing a string into a timezone aware datetime object

##############################################################################
Python 3.2+ has support for %z format when parsing a string into a datetime object.
UTC offset in the form +HHMM or -HHMM (empty string if the object is naive).
Python 3.x Version
≥ 3.2


++++++++++++++++++++++++++++++++++++++++++++++++++
=================================================

import datetime
dt = datetime.datetime.strptime("2016-04-15T08:27:18-0500","%Y-%m-%dT%H:%M:%S%z")

In [2]:
For other versions of Python, you can use an external library such as dateutil,which makes
parsing a string with timezone into a datetime object is quick.
####################################################################################
import dateutil.parser
dt = dateutil.parser.parse("2016-04-15T08:27:18-0500")
The dt variable is now a datetime object with the following value:
datetime.datetime(2016, 4, 15, 8, 27, 18, tzinfo=tzoffset(None, -18000))

2016-04-15 08:27:18-05:00


Section 5.2: Constructing timezone-aware datetimes
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

By default all datetime objects are naive. To make them timezone-aware, you must attach a tzinfo object, which provides the UTC offset and timezone abbreviation as a function of date and time.

++++++++++++++++++++++++++++++++++++++++++++++++++
==================================================

Fixed Offset Time Zones
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
For time zones that are a fixed offset from UTC, in Python 3.2+, the datetime module provides the timezone class,a concrete implementation of tzinfo , which takes a timedelta and an (optional) name parameter:

++++++++++++++++++++++++++++++++++++++++++++++++++
==================================================
Python 3.x Version
≥ 3.2
from datetime import datetime, timedelta, timezone
JST = timezone(timedelta(hours=+9))
dt = datetime(2015, 1, 1, 12, 0, 0, tzinfo=JST)
print(dt)

# 2015-01-01 12:00:00+09:00
print(dt.tzname())

# UTC+09:00
dt = datetime(2015, 1, 1, 12, 0, 0, tzinfo=timezone(timedelta(hours=9), 'JST'))
print(dt.tzname)

# 'JST

In [16]:
from datetime import datetime, timedelta
import pytz
PT = pytz.timezone('Asia/Kolkata')
#print(PT)
dt_pdt = PT.localize(datetime(2021, 4, 1, 0, 30))
print(dt)
##########################


2016-04-15 08:27:18-05:00


In [18]:
from datetime import datetime
from pytz import timezone

fmt = "%Y-%m-%d %H:%M:%S %Z%z"

# get the current time in UTC
now_utc = datetime.now(timezone('utc'))
print('UTC :', now_utc)

# # Convert this to EST
# now_est = now_utc.astimezone(timezone('US/Eastern'))
# print('EST :', now_est)

# now_berlin = now_utc.astimezone(timezone('Europe/Berlin'))
# print('Berlin:', now_berlin)

# now_london = now_utc.astimezone(timezone('Europe/London'))
# print('London:', now_london)

# now_Kolkata = now_utc.astimezone(timezone('Asia/Kolkata'))
# print('Kolkata:', now_Kolkata)

# now_Shanghai = now_utc.astimezone(timezone('Asia/Shanghai'))
# print('Shanghai:',now_Shanghai)

# now_Sydney = now_utc.astimezone(timezone('Australia/Sydney'))
# print('Sydney:',now_Sydney)

# now_Johannesburg = now_utc.astimezone(timezone('Africa/Johannesburg'))
# print('Johannesburg:',now_Johannesburg)

UTC : 2021-02-11 09:42:50.877201+00:00


In [ ]:
For Python versions before 3.2, it is necessary to use a third party library, such as 
dateutil . dateutil provides an equivalent class, tzoffset,which (as of version 2.5.3) 
takes arguments of the form dateutil.tz.tzoffset(tzname, offset) , where offset is 
specified in seconds:
    
###################################################################################
Python 3.x Version
< 3.2

Python 2.x Version
< 2.7
#################################################################################
from datetime import datetime, timedelta
from dateutil import tz
JST = tz.tzoffset('JST', 9 * 3600) # 3600 seconds per hour
dt = datetime(2015, 1, 1, 12, 0, tzinfo=JST)
print(dt)
# 2015-01-01 12:00:00+09:00
print(dt.tzname)
# 'JST'

In [ ]:
Zones with daylight savings time

##############################################################################
For zones with daylight savings time, python standard libraries do not provide a standard 
class, so it is necessary to use a third party library. pytz and dateutil are popular 
libraries providing time zone classes.

##################################################################################
In addition to static time zones, dateutil provides time zone classes that use daylight 
savings time (see the documentation for the tz module). You can use the tz.gettz() method 
to get a time zone object, which can then be passed directly to the datetime constructor:
    
######################################################################################
from datetime import datetime
from dateutil import tz
local = tz.gettz() # Local time
PT = tz.gettz('US/Pacific') # Pacific time
dt_l = datetime(2015, 1, 1, 12, tzinfo=local) # I am in EST
dt_pst = datetime(2015, 1, 1, 12, tzinfo=PT)
dt_pdt = datetime(2015, 7, 1, 12, tzinfo=PT) # DST is handled automatically
print(dt_l)
# 2015-01-01 12:00:00-05:00
print(dt_pst)
# 2015-01-01 12:00:00-08:00
print(dt_pdt)
# 2015-07-01 12:00:00-07:00

########################################################################################
CAUTION: As of version 2.5.3, dateutil does not handle ambiguous datetimes correctly, and 
will always default to the later date. There is no way to construct an object with a 
dateutil timezone representing, for example 2015-11-01 1:30 EDT-4 , since this is during a
daylight savings time transition.

All edge cases are handled properly when using pytz , but pytz time zones should not be directly attached to time
zones through the constructor. Instead, a pytz time zone should be attached using the time zone's localize
method:
    
###################################################################################
from datetime import datetime, timedelta
import pytz
PT = pytz.timezone('US/Pacific')
dt_pst = PT.localize(datetime(2015, 1, 1, 12))
dt_pdt = PT.localize(datetime(2015, 11, 1, 0, 30))
print(dt_pst)
# 2015-01-01 12:00:00-08:00
print(dt_pdt)
# 2015-11-01 00:30:00-07:00
Be aware that if you perform datetime arithmetic on a pytz -aware time zone, you must 
either perform the calculations in UTC (if you want absolute elapsed time), or you must 
call normalize() on the result

In [ ]:
Section 5.3: Computing time differences
    
###################################################################################
the timedelta module comes in handy to compute differences between times:
    
#############################################################################
from datetime import datetime, timedelta
now = datetime.now()
then = datetime(2016, 5, 23)
# datetime.datetime(2016, 05, 23, 0, 0, 0)
Specifying time is optional when creating a new datetime object
delta = now-then
delta is of type timedelta
print(delta.days)
# 60
print(delta.seconds)
# 40826

#####################################################################################
To get n day's after and n day's before date we could use:
n day's after date:

def get_n_days_after_date(date_format="%d %B %Y", add_days=120):
    date_n_days_after = datetime.datetime.now() + timedelta(days=add_days)
    return date_n_days_after.strftime(date_format)

##################################################################################
n day's before date:


def get_n_days_before_date(self, date_format="%d %B %Y", days_before=120):
    date_n_days_ago = datetime.datetime.now() - timedelta(days=days_before)
    return date_n_days_ago.strftime(date_format)

##################################################################################

In [ ]:
Section 5.5: Switching between time zones
    
####################################################################################
To switch between time zones, you need datetime objects that are timezone-aware.

################################################################################
from datetime import datetime
from dateutil import tz
utc = tz.tzutc()
local = tz.tzlocal()
utc_now = datetime.utcnow()
utc_now # Not timezone-aware.
utc_now = utc_now.replace(tzinfo=utc)
utc_now # Timezone-aware.
local_now = utc_now.astimezone(local)
local_now # Converted to local time.

In [ ]:
Section 5.6: Simple date arithmetic
    
#################################################################################
Dates don't exist in isolation. It is common that you will need to find the amount of time 
between dates or determine what the date will be tomorrow. This can be accomplished using 
timedelta objects

##################################################################################
import datetime
today = datetime.date.today()
print('Today:', today)
yesterday = today - datetime.timedelta(days=1)
print('Yesterday:', yesterday)
tomorrow = today + datetime.timedelta(days=1)
print('Tomorrow:', tomorrow)
print('Time between tomorrow and yesterday:', tomorrow - yesterday)

This will produce results similar to:
Today: 2016-04-15
Yesterday: 2016-04-14
Tomorrow: 2016-04-16
Difference between tomorrow and yesterday: 2 days, 0:00:00

In [ ]:
Section 5.7: Converting timestamp to datetime
The datetime module can convert a POSIX timestamp to a ITC datetime object.
The Epoch is January 1st, 1970 midnight.
import time
from datetime import datetime
seconds_since_epoch=time.time()
#1469182681.709
utc_date=datetime.utcfromtimestamp(seconds_since_epoch) #datetime.datetime(2016, 7, 22, 10, 18, 1,
709000)

In [ ]:
Section 5.12: Fuzzy datetime parsing (extracting datetime out
of a text)
    
#########################################################################################
It is possible to extract a date out of a text using the dateutil parser in a "fuzzy" mode,
where components of the string not recognized as being part of a date are ignored.

#################################################################################
from dateutil.parser import parse
dt = parse("Today is January 1, 2047 at 8:21:00AM", fuzzy=True)
print(dt)
dt is now a datetime object and you would see datetime.datetime(2047, 1, 1, 8, 21) printed.

In [ ]:
Section 5.13: Iterate over dates

######################################################################################
Sometimes you want to iterate over a range of dates from a start date to some end date. 
You can do it using datetime library and timedelta object:

###############################################################################    
import datetime
# The size of each step in days
day_delta = datetime.timedelta(days=1)
start_date = datetime.date.today()
end_date = start_date + 7*day_delta
for i in range((end_date - start_date).days):
print(start_date + i*day_delta)

################################################################################
Which produces:
2016-07-21
2016-07-22
2016-07-23
2016-07-24
2016-07-25
2016-07-26
2016-07-27

In [3]:
import datetime
dt = datetime.datetime.strptime("2016-04-15T08:27:18-0500", "%Y-%m-%dT%H:%M:%S%z")
print(dt)

2016-04-15 08:27:18-05:00
